In [1]:
##########################
#  Code Author: 1359531  #
##########################

In [2]:
# import nltk
# nltk.download('punkt')

In [3]:
from nlpfunctions import bow
from nlpfunctions import clean_up_sentence
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

In [4]:
import json
import random
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout

Using TensorFlow backend.


In [5]:
# import our heimdall intents file
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [6]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

41 documents
13 classes ['debug', 'debugoff', 'free', 'gen_cert', 'gen_cert_type_rg', 'gen_cert_type_vm', 'goodbye', 'greeting', 'hay', 'master', 'name', 'purpose', 'thanks']
53 unique stemmed words ["'s", 'anyon', 'ar', 'bye', 'can', 'cert', 'cre', 'day', 'debug', 'diagnost', 'dis', 'do', 'en', 'fath', 'fre', 'gen', 'good', 'goodby', 'group', 'hello', 'help', 'hi', 'hour', 'how', 'is', 'lat', 'machin', 'mad', 'mak', 'me', 'mod', 'nam', 'off', 'on', 'purpos', 'resourc', 'rg', 'see', 'start', 'stop', 'tel', 'thank', 'that', 'ther', 'today', 'virt', 'vm', 'what', 'when', 'who', 'work', 'yo', 'you']


In [7]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])
train_x = np.asarray(train_x, dtype=np.float32)
train_y = np.asarray(train_y, dtype=np.float32)

In [8]:
model = Sequential()
model.add(Dense(48, input_dim=train_x.shape[1], activation='relu'))
# model.add(Dropout(0.1))
model.add(Dense(32, activation='relu'))
# model.add(Dropout(0.5))
model.add(Dense(16, activation='relu'))
# model.add(Dropout(0.5))
model.add(Dense(train_y.shape[1], activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [9]:
model.fit(train_x, train_y, epochs=200)

Epoch 1/200
41/41 [==============================] - 0s 5ms/step - loss: 0.2706 - accuracy: 0.9231
Epoch 2/200
41/41 [==============================] - 0s 276us/step - loss: 0.2692 - accuracy: 0.9231
Epoch 3/200
41/41 [==============================] - 0s 264us/step - loss: 0.2682 - accuracy: 0.9231
Epoch 4/200
41/41 [==============================] - 0s 333us/step - loss: 0.2672 - accuracy: 0.9231
Epoch 5/200
41/41 [==============================] - 0s 303us/step - loss: 0.2662 - accuracy: 0.9231
Epoch 6/200
41/41 [==============================] - 0s 256us/step - loss: 0.2652 - accuracy: 0.9231
Epoch 7/200
41/41 [==============================] - 0s 269us/step - loss: 0.2642 - accuracy: 0.9231
Epoch 8/200
41/41 [==============================] - 0s 279us/step - loss: 0.2632 - accuracy: 0.9231
Epoch 9/200
41/41 [==============================] - 0s 227us/step - loss: 0.2624 - accuracy: 0.9231
Epoch 10/200
41/41 [==============================] - 0s 199us/step - loss: 0.2614 - accuracy

41/41 [==============================] - 0s 276us/step - loss: 0.0935 - accuracy: 0.9719
Epoch 82/200
41/41 [==============================] - 0s 288us/step - loss: 0.0908 - accuracy: 0.9719
Epoch 83/200
41/41 [==============================] - 0s 266us/step - loss: 0.0880 - accuracy: 0.9719
Epoch 84/200
41/41 [==============================] - 0s 122us/step - loss: 0.0852 - accuracy: 0.9737
Epoch 85/200
41/41 [==============================] - 0s 275us/step - loss: 0.0824 - accuracy: 0.9756
Epoch 86/200
41/41 [==============================] - 0s 229us/step - loss: 0.0797 - accuracy: 0.9775
Epoch 87/200
41/41 [==============================] - 0s 178us/step - loss: 0.0770 - accuracy: 0.9775
Epoch 88/200
41/41 [==============================] - ETA: 0s - loss: 0.0810 - accuracy: 0.97 - 0s 332us/step - loss: 0.0743 - accuracy: 0.9775
Epoch 89/200
41/41 [==============================] - 0s 255us/step - loss: 0.0718 - accuracy: 0.9775
Epoch 90/200
41/41 [==============================] -

In [10]:
# evaluate the keras model
_, accuracy = model.evaluate(train_x, train_y)
print('Accuracy: %.2f' % (accuracy*100))

41/41 [==============================] - 0s 837us/step
Accuracy: 100.00


In [11]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 48)                2592      
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1568      
_________________________________________________________________
dense_3 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_4 (Dense)              (None, 13)                221       
Total params: 4,909
Trainable params: 4,909
Non-trainable params: 0
_________________________________________________________________


In [12]:
# save all of our data structures
pickle.dump( {'words':words, 'classes':classes}, open( "meta.pkl", "wb" ) )
model.save("model.h5")

In [13]:
#Test
thres = 0
p = np.array([bow("how are you", words)])
results = model.predict(p)
for c,p in zip(classes,results[0]):
    if(p>thres):
        print(c,p)

debug 0.00046151897
debugoff 6.231448e-05
free 0.0059284857
gen_cert 4.8582428e-05
gen_cert_type_rg 8.076307e-05
gen_cert_type_vm 0.0006012311
goodbye 0.00014164616
greeting 0.00048511918
hay 0.99175197
master 8.618967e-06
name 0.00029560248
purpose 9.143533e-05
thanks 4.2763262e-05


In [14]:
#Classifier Test
p = np.array([bow("Who created you", words)])
classes[int(model.predict_classes(p))]

'master'

In [15]:
thres = 0
p = np.array([bow("can you please generate certificate", words)])
results = model.predict(p)
for c,p in zip(classes,results[0]):
    if(p>thres):
        print(c,p)

debug 3.6625324e-06
debugoff 0.000582685
free 2.1391547e-07
gen_cert 0.99758935
gen_cert_type_rg 2.390146e-07
gen_cert_type_vm 8.002922e-11
goodbye 0.00042443073
greeting 0.00013447402
hay 1.4769495e-09
master 1.5773336e-05
name 2.0558773e-05
purpose 0.0012260259
thanks 2.5214695e-06


In [16]:
# #Classifier Test
# p = np.array([bow("how are you", words)])
# classes[int(model.predict_classes(p))]

In [17]:
thres = 0
p = np.array([bow("vm", words)])
results = model.predict(p)
for c,p in zip(classes,results[0]):
    if(p>thres):
        print(c,p)

debug 0.0001147208
debugoff 4.1952342e-05
free 6.192921e-05
gen_cert 1.1185695e-05
gen_cert_type_rg 4.6014156e-05
gen_cert_type_vm 0.981058
goodbye 6.0464913e-07
greeting 0.015369388
hay 0.00034031246
master 5.120244e-07
name 2.3535363e-06
purpose 5.0946535e-05
thanks 0.0029020119


In [18]:
results

array([[1.1472080e-04, 4.1952342e-05, 6.1929211e-05, 1.1185695e-05,
        4.6014156e-05, 9.8105800e-01, 6.0464913e-07, 1.5369388e-02,
        3.4031246e-04, 5.1202443e-07, 2.3535363e-06, 5.0946535e-05,
        2.9020119e-03]], dtype=float32)